# ---WORK IN PROGRESS---

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

A little helper to annotate numbers on bar plots

In [ ]:
def annotate_bar_plot(plot):
    """
    Source: https://www.geeksforgeeks.org/how-to-annotate-bars-in-barplot-with-matplotlib-in-python/
    """
    for bar in plot.patches: 
        plot.annotate(text=f'{bar.get_height():.2f}', xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                      ha='center', va='center', size=12, xytext=(0, 8), textcoords='offset points')
        
def annotate_boxplot_median(medians, plot, offset):
    """
    Source: https://stackoverflow.com/questions/38649501/labeling-boxplot-in-seaborn-with-median-value
    """
    vertical_offset = np.median(medians) * offset
    
    for xtick in plot.get_xticks():
        plot.text(x=xtick, y=medians[xtick] + vertical_offset, s=medians[xtick], 
                  horizontalalignment='center', fontsize=12, weight='semibold')

Quick look over the data, ignoring those not having Star rating

In [ ]:
ramen = pd.read_csv('/kaggle/input/ramen-ratings/ramen-ratings.csv')
ramen.drop(['Review #'], axis=1, inplace=True)
ramen = ramen.query('Stars != "Unrated"')
ramen['Stars'] = ramen['Stars'].astype(float)
ramen.shape

In [ ]:
ramen.head()

# Package style

In [ ]:
ramen['Style'].value_counts()

In [ ]:
ramen['Style'] = ramen['Style'].apply(lambda x: 'Box/Can/Bar' if x in ['Box', 'Can', 'Bar'] else x)

In [ ]:
style = ramen[['Style', 'Stars']].groupby('Style').mean().reset_index()


plt.figure(figsize=(14, 7))
plot = sns.barplot(x='Style', y='Stars', data=style.sort_values(by='Stars', ascending=False))
annotate_bar_plot(plot)
plt.title('Average rating by packing style', fontsize=15)
plt.show()

# Country

No surprise Japan is the biggest manufacturer!!

In [ ]:
count = ramen['Country'].value_counts()

plt.figure(figsize=(14, 10))
plt.pie(count, labels=count.index.tolist(), autopct='%1.1f%%', startangle=90, counterclock=False)
plt.title('Country distribution', fontsize=18)
plt.show()

In [ ]:
country = ramen[['Country', 'Stars']].groupby('Country').mean().reset_index()

plt.figure(figsize=(21, 7))
plot = sns.barplot(x='Country', y='Stars', data=country.sort_values(by='Stars', ascending=False))
annotate_bar_plot(plot)
plt.title('Average rating by country', fontsize=15)
plt.xticks(rotation=30)
plt.show()

# Brand

A brand can have many variations, the dictionary below unifies such cases

In [ ]:
name_correction = {
    'Nissin': ['7 Select', '7 Select/Nissin', 'Nissin'],
    'Acecook': ['Acecook', 'Vina Acecook'],
    'ChoripDong': ['Chorip Dong', 'ChoripDong'],
    'Fashion Food': ['Fashion Food', 'Fashion Foods'],
    'Goku-Uma': ['Goku Uma', 'Goku-Uma'],
    'Hua Feng': ['Hua Feng', 'Hua Feng Noodle Expert'],
    'KOKA': ['KOKA', 'Koka'],
    'Lishan': ['Lishan', 'Lishan Food Manufacturing'],
    'MAMA': ['MAMA', 'Mama'],
    'Paldo': ['Paldo', 'Paldo Vina'],
    'Prima': ['Prima', 'Prima Taste'],
    'Sakurai': ['Sakurai', 'Sakurai Foods'],
    'Samyang': ['Samyang', 'Samyang Foods'],
    'Baijia': ['Baijia', 'Sichuan Baijia'],
    'Takamori': ['Takamori', 'Takamori Kosan'],
    'Unif': ['Unif', 'Unif / Tung-I', 'Unif Tung-I', 'Unif-100'],
    'Ve Wong': ['A-One', 'Saigon Ve Wong', 'Ve Wong'],
    'Wei Lih': ['Weh Lih', 'Wei Lih'],
    'Wu Mu': ['Wu Mu', 'Wu-Mu']
}

In [ ]:
ramen['Brand_Revised'] = ramen['Brand']

for index, row in ramen.iterrows():
    for key, value in name_correction.items():
        if row['Brand'] in value:
            ramen.loc[index, 'Brand_Revised'] = key

In [ ]:
brand_count = ramen['Brand_Revised'].value_counts()
brand_count = brand_count[brand_count >= 10].index.tolist()

brand = ramen.query(f'Brand_Revised == {brand_count}')[['Brand_Revised', 'Stars']].groupby('Brand_Revised').mean().reset_index()

plt.figure(figsize=(28, 7))
plot = sns.barplot(x='Brand_Revised', y='Stars', data=brand.sort_values(by='Stars', ascending=False))
annotate_bar_plot(plot)
plt.title('Average rating by Brand', fontsize=15)
plt.xticks(rotation=30)
plt.show()

# Variety

Most information shall be derived from this column


## Word count

Find words with high frequency. Values in Variety are short, condense description of the ramen, so there aren't that many unique words.

In [ ]:
# import string

# word_freq = ramen['Variety'].apply(lambda x: x.lower().translate(str.maketrans('', '', string.punctuation)))
# word_freq = word_freq.str.split(expand=True).stack().value_counts().reset_index()
# word_freq.columns = ['word', 'count']
# word_freq.to_csv('word_freq.csv', index=False)

word_freq = pd.read_csv('/kaggle/input/ramen-dataset-word-count/word_freq.csv')
word_freq = word_freq.query('category != "other"')
word_freq.shape

Each word is classified into different categories

In [ ]:
word_freq['category'].value_counts()

## Cuisine

This may not be the same as Country column, as a country can still make ramen having other country's flavour.

Similarly, there are several variations for one cuisine. Other than different English forms, there are words in other languages as well.

Foreign language lesson anyone?
* **Japanese:** Japanische (German), Jepeng (Indonesian?), Nippon (Japanese)
* **Singaporean:** Singapura (Indonesian)
* **Western:** Yousyoku (meaning "Western food" in Japanese)

In [ ]:
cuisine = word_freq.query('category == "cuisine"')['word'].tolist()

cuisine_correction = {
    'indonesian': ['indonesia', 'indonesian'],
    'japanese': ['japan', 'japanische', 'jepeng', 'nippon'],
    'korean': ['korea', 'korean'],
    'malaysian': ['malaysia', 'malaysian'],
    'chinese': ['mandarin', 'chinese'],
    'singaporean': ['singaporean', 'singapura', 'singapore'],
    'taiwanese': ['taiwan', 'taiwanese'],
    'thai': ['thai', 'thailand'],
    'western': ['yousyoku'],
    'vietnamese': ['vietnam', 'vietnamese']
}

Does mentioning a particular cuisine yield higher rating?

In [ ]:
ramen['Cuisine'] = ramen['Variety'].apply(lambda x: list(set(x.lower().split()) & set(cuisine)))
ramen['Has_Cuisine'] = (ramen['Cuisine'].map(len) > 0).astype(int)
ramen['Has_Cuisine'].value_counts()

In [ ]:
brand = ramen[['Has_Cuisine', 'Stars']].groupby('Has_Cuisine').mean().reset_index()

plt.figure(figsize=(7, 7))
plot = sns.barplot(x='Has_Cuisine', y='Stars', data=brand.sort_values(by='Stars'))
annotate_bar_plot(plot)
plt.title('Variety mentioning Cuisine vs not mentioning Cuisine', fontsize=15)
plt.show()

Apparently no, adding just any cuisine in the description doesn't make a ramen good. It has to be the right one.

In [ ]:
cuisine_df = ramen.query('Has_Cuisine == 1')[['Cuisine', 'Stars']]
cuisine_df = cuisine_df.explode('Cuisine')
cuisine_df.shape

In [ ]:
for key, value in cuisine_correction.items():
    cuisine_df['Cuisine'].replace(to_replace=value, value=key, inplace=True)

In [ ]:
fig, ax = plt.subplots(2, 1, constrained_layout=True, figsize=(28, 15))

count = cuisine_df['Cuisine'].value_counts()
ax[0].pie(count, labels=count.index.tolist(), autopct='%1.1f%%', startangle=90, counterclock=False, textprops={'fontsize': 15})
ax[0].set_title('Cuisine distribution', fontsize=18)

data = cuisine_df.groupby('Cuisine').mean().reset_index()
data.sort_values(by='Stars', ascending=False, inplace=True)

sns.barplot(x='Cuisine', y='Stars', data=data, ax=ax[1])
annotate_bar_plot(ax[1])
ax[1].set_title('Average Stars for each Cuisine', fontsize=15)
ax[1].tick_params(labelsize=15)

plt.show()

Vietnamese is in top 5 !!

## Colour

In [ ]:
colours_list = word_freq.query('category == "colour"')['word'].tolist()
colours_list = [x for x in colours_list if x != 'aka']
print(sorted(colours_list))

In [ ]:
colours_correction = {
    'red': ['akai', 'czerwony'],
    'black': ['dark', 'hitam', 'kuro'],
    'yellow': ['gold', 'golden'],
    'green': ['green', 'groene', 'hijau', 'midori'],
    'white': ['shiroi']
}

In [ ]:
colours_df = ramen[['Variety', 'Stars']]
colours_df['Colour'] = colours_df['Variety'].apply(lambda x: list(set(x.lower().split()) & set(colours_list)))

In [ ]:
data = colours_df[['Colour', 'Stars']]
data['Has_Colour'] = (data['Colour'].map(len) > 0).astype(int)
data = data.groupby('Has_Colour').mean().reset_index()

plt.figure(figsize=(7, 7))
plot = sns.barplot(x='Has_Colour', y='Stars', data=data.sort_values(by='Stars'))
annotate_bar_plot(plot)
plt.title('Variety mentioning Colour vs not mentioning Colour', fontsize=15)
plt.show()

In [ ]:
colours_df = colours_df.explode('Colour').dropna()

for key, value in colours_correction.items():
    colours_df['Colour'].replace(to_replace=value, value=key, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(28, 10))

count = colours_df['Colour'].value_counts()
ax[0].pie(count, labels=count.index.tolist(), autopct='%1.1f%%', startangle=90, counterclock=False, textprops={'fontsize': 15})
ax[0].set_title('Colour distribution', fontsize=18)

data = colours_df[['Colour', 'Stars']].groupby('Colour').mean().reset_index()
data.sort_values(by='Stars', ascending=False, inplace=True)

sns.barplot(x='Colour', y='Stars', data=data, ax=ax[1], palette=data['Colour'].values.tolist())
annotate_bar_plot(ax[1])
ax[1].set_title('Average Stars for each Type', fontsize=15)
ax[1].tick_params(labelsize=15)

ax[1].set_facecolor('gray')

plt.show()

## Type of ramen

In [ ]:
ramen_type = word_freq.query('category == "type"')['word'].tolist()
ramen_type = [x for x in ramen_type if x not in ['puri', 'tenobe', 'demae']]
print(sorted(ramen_type))

In [ ]:
type_correction = {
    'grain': ['barley', 'buckwheat', 'grains', 'millet', 'oat', 'wheat', 'wholegrain'],
    'vermicelli': ['beehoon', 'bihon', 'bihun', 'bun', 'mien', 'sotanghon', 'vermicelli'],
    'pasta': ['bucatini', 'fettuccine', 'fettucini', 'fideos', 'macaroni', 'pasta'],
    'rice': ['com', 'khao', 'porridge', 'rice', 'ricey'],
    'snack': ['crackers', 'mochi', 'snack', 'tortilla'],
    'ramen': ['lamen', 'mee', 'mein', 'men', 'mi', 'mian', 'mie', 'myon', 'myun', 'noodle', 'noodles', 'noodlez',
              'noolde', 'nooldes', 'nudelsuppe', 'nudlar', 'ramen', 'ramyon', 'ramyun'],
    'special': ['chukasoba', 'futomen', 'soba', 'somen', 'udon', 'udong']
}

In [ ]:
type_df = ramen[['Variety', 'Stars']]
type_df['Type'] = type_df['Variety'].apply(lambda x: list(set(x.lower().split()) & set(ramen_type)))

In [ ]:
type_df = type_df.explode('Type').dropna()

for key, value in type_correction.items():
    type_df['Type'].replace(to_replace=value, value=key, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(28, 10))

count = type_df['Type'].value_counts()
ax[0].pie(count, labels=count.index.tolist(), autopct='%1.1f%%', startangle=90, counterclock=False, textprops={'fontsize': 15})
ax[0].set_title('Ramen Type distribution', fontsize=18)

data = type_df[['Type', 'Stars']].groupby('Type').mean().reset_index()
data.sort_values(by='Stars', ascending=False, inplace=True)

sns.barplot(x='Type', y='Stars', data=data, ax=ax[1])
annotate_bar_plot(ax[1])
ax[1].set_title('Average Stars for each Type', fontsize=15)
ax[1].tick_params(labelsize=15)

plt.show()

## Cooking method

In [ ]:
methods = word_freq.query('category == "method"')['word'].tolist()
methods = [x for x in methods if x not in ['saab', 'sabb']]
print(sorted(methods))

In [ ]:
method_correction = {
    'grill': ['assado', 'baked', 'barbecue', 'barbeque', 'bbq', 'grilled', 'roast', 'roasted', 'smoked', 'sunbaked', 'toasted'],
    'dry': ['bibim', 'dried', 'dry', 'mix', 'mixed', 'precooked', 'soupless'],
    'fry': ['fry', 'fried', 'goreng', 'pan', 'stir', 'stirfried', 'stirfry'],
    'cook': ['braised', 'simmered', 'stew', 'stewde', 'stewed'],
    'steam': ['steam', 'steamed'],
    'veggie': ['chay', 'vegan', 'vegetarian'],
    'cold': ['cold', 'frozen'],
    'cut': ['crushed', 'minced'],
    'other': ['pickled', 'nonfried', 'unfried']
}

In [ ]:
method_df = ramen[['Variety', 'Stars']]
method_df['Method'] = method_df['Variety'].apply(lambda x: list(set(x.lower().split()) & set(methods)))
method_df = method_df.explode('Method').dropna()

for key, value in method_correction.items():
    method_df['Method'].replace(to_replace=value, value=key, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(28, 10))

count = method_df['Method'].value_counts()
ax[0].pie(count, labels=count.index.tolist(), autopct='%1.1f%%', startangle=90, counterclock=False, textprops={'fontsize': 15})
ax[0].set_title('Method distribution', fontsize=18)

data = method_df[['Method', 'Stars']].groupby('Method').mean().reset_index()
data.sort_values(by='Stars', ascending=False, inplace=True)

sns.barplot(x='Method', y='Stars', data=data, ax=ax[1])
annotate_bar_plot(ax[1])
ax[1].set_title('Average Stars for each Method', fontsize=15)
ax[1].tick_params(labelsize=15)

plt.show()

## Flavour

In [ ]:
flavour = word_freq.query('category == "flavour"')['word'].tolist()
# flavour = [x for x in methods if x not in ['saab', 'sabb']]
print(sorted(flavour))

In [ ]:
flavour_correction = {
    'sour': ['asam', 'assam', 'chua', 'sour'],
    'spicy': ['berapi', 'bul', 'burning', 'caldo', 'cay', 'fire', 'flamin', 'gekikara', 'hot', 'karakuchi', 'picante', 'spicier', 'spicy'],
    'fatty': ['cheezy', 'creamy', 'fat', 'rich', 'nutty'],
    'savoury': ['salty', 'gurih', 'savoury', 'umami'],
    'sweet': ['sweet', 'fruity'],
    'light': ['light', 'mild'],
    'other': ['chilled', 'matcha']
}

In [ ]:
flavour_df = ramen[['Variety', 'Stars']]
flavour_df['Flavour'] = flavour_df['Variety'].apply(lambda x: list(set(x.lower().split()) & set(flavour)))
flavour_df = flavour_df.explode('Flavour').dropna()

for key, value in flavour_correction.items():
    flavour_df['Flavour'].replace(to_replace=value, value=key, inplace=True)

In [ ]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(28, 10))

count = flavour_df['Flavour'].value_counts()
ax[0].pie(count, labels=count.index.tolist(), autopct='%1.1f%%', startangle=90, counterclock=False, textprops={'fontsize': 15})
ax[0].set_title('Flavour distribution', fontsize=18)

data = flavour_df[['Flavour', 'Stars']].groupby('Flavour').mean().reset_index()
data.sort_values(by='Stars', ascending=False, inplace=True)

sns.barplot(x='Flavour', y='Stars', data=data, ax=ax[1])
annotate_bar_plot(ax[1])
ax[1].set_title('Average Stars for each Flavour', fontsize=15)
ax[1].tick_params(labelsize=15)

plt.show()

In [ ]:
word_freq['category'].value_counts()